In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
from google.colab import drive

In [5]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [13]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [17]:
ls data

men_shoes.csv  shoe_prices.csv


In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].factorize()[0]

In [20]:
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [21]:
model = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [22]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
model = RandomForestRegressor(max_depth=5, n_estimators=100,random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [23]:
test = {'key': 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [0]:
def parse_features(x):
  if str(x) == 'nan': return []
  return literal_eval(x.replace('\\"','"'))

df['features_parsed'] = df['features'].map(parse_features)

In [28]:
df['features_parsed'].head().values

array([list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in pounds)', 'value': ['0.45']}, {'key': 'Condition', 'value': ['New']}, {'key': 'Brand', 'value': ['SERVUS BY HONEYWELL']}, {'key': 'manufacturer_part_number', 'value': ['ZSR101BLMLG']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [30]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [31]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [32]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [33]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_net weight', 'feat_temple size:', 'feat_hairstyle',
       'feat_pattern', 'feat_season', 'feat_item width', 'feat_rx-able',
       'feat_frame color', 'feat_euro size', 'feat_hood'],
      dtype='object', length=526)

In [34]:
df[df['feat_athlete'].isnull()].shape

(18272, 526)

In [35]:
df.shape[0]

18280

In [37]:
df[False == df['feat_athlete'].isnull()].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [42]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [52]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[df.brand == df.feat_brand][ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
0,josmo,josmo
1,josmo,josmo
2,servus by honeywell,servus by honeywell
3,servus by honeywell,servus by honeywell
4,servus by honeywell,servus by honeywell


In [54]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[df.brand == df.feat_brand].shape

(8846, 531)

In [0]:
feats = ['']

In [56]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.29135891886058, 4.259905572498513)

In [82]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_fabrication',
 'feat_certifications and listings',
 'feat_catalog',
 'feat_shoe category',
 'feat_location - city/state',
 'feat_recommended location',
 'feat_multi pack indicator',
 'feat_location - country',
 'feat_clothing category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_label_cat',
 'feat_sock style_cat',
 'feat_resolution megapixels_cat',
 'feat_work shoes_cat',
 'feat_navy blue_cat',
 'feat_leather grade_cat',
 'feat_kids backpacks_cat',
 'feat_case back_cat',
 'feat_watch style_cat',
 'feat_casing_cat',
 'feat_fabrication_cat',
 'feat_fits brands_cat',
 'feat_item weight_cat',
 'feat_model number_cat',
 'feat_measurements:_cat',
 'feat_country//region of manufacture_cat',
 'feat_issued/ not-issued_cat',
 'feat_box_cat',
 'feat_metal type_cat',
 'feat_rise_cat',
 'feat_main colour_cat',
 'feat_productgroup_cat',
 'feat_er

In [74]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.88957429416198, 5.084174872899088)

In [92]:
X = df[feats].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

(-57.168918308995885, 4.373934643161059)


Weight,Feature
0.2617 ± 0.0108,brand_cat
0.1071 ± 0.0145,feat_material_cat
0.0239 ± 0.0034,feat_gender_cat
0.0217 ± 0.0015,feat_brand_cat
0.0135 ± 0.0008,feat_shape_cat
0.0041 ± 0.0007,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [0]:
feats = ['brand_cat', 'feat_shape_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
feats += {}
#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [67]:
df['brand'].value_counts()

nike              1777
puma               609
ralph lauren       526
vans               386
new balance        371
                  ... 
sunglasses           1
adeyro               1
bugatchi             1
puentes denver       1
basics               1
Name: brand, Length: 1732, dtype: int64

In [68]:
df['brand'].value_counts(normalize=True)

nike              0.097210
puma              0.033315
ralph lauren      0.028775
vans              0.021116
new balance       0.020295
                    ...   
sunglasses        0.000055
adeyro            0.000055
bugatchi          0.000055
puentes denver    0.000055
basics            0.000055
Name: brand, Length: 1732, dtype: float64

In [78]:
df[df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [79]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'season': 'all-season', 'occasion': 'athletic', 'material': 'synthetic', 'gender': 'men', 'shoe size': '11.5', 'color': 'pink', 'manufacturer part number': '683705 801', 'brand': 'nike', 'age group': 'adult', 'size': '12'},
       {'country/region of manufacture': 'china', 'mpn': '684860-003', 'color': 'silver'},
       {'sport': 'basketball shoes', 'style': 'basketball shoes', 'pattern': 'solid', 'occasion': 'basketball shoes', 'material': 'mesh', 'gender': 'men', 'shoe size': '11 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'country/region of manufacture': 'china', 'fabric content': 'mesh', 'color': 'other', 'model': '705370 484', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'basketball shoes', 'manufacturer part number': '705370 484', 'condition': 'new with box', 'variant group id': '224#mp#705370 484', 'brand': 'nike', 'age group': 'men', 'shoe width': 'd(m)'},
       {'shoe size': '12', 'gende

In [89]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [93]:
!git add matrix_one/day5.ipynb

fatal: pathspec 'matrix_one/day5.ipynb' did not match any files


In [94]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md
